# Using Bedrock ModelAnalyzer

In [ ]:
import pickle

import numpy as np
import pandas as pd
import lightgbm as lgb
from bedrock_client.bedrock.analyzer.model_analyzer import ModelAnalyzer
from bedrock_client.bedrock.analyzer import ModelTypes

from preprocess.constants import FEATURES, TARGET, CONFIG_FAI

##  Load model and test dataset

Here we use our model artefact trained elsewhere.

In [ ]:
model = pickle.load(open("output/lgb_model.pkl", "rb"))

In [ ]:
test_df = pd.read_parquet("output/test.gz.parquet")
x_test = test_df[FEATURES]
y_test = test_df[TARGET].values

In [ ]:
# Predict
y_pred = model.predict(x_test)

## Explainability and Fairness using ModelAnalyzer

We shall use `bedrock_client.bedrock.analyzer.model_analyzer.ModelAnalyzer` to generate model explainability analysis and fairness metrics locally. You can use it to generate and log them in Bedrock.

Refer to [documentation](https://docs.basis-ai.com/guides/explainability-and-fairness) and [API references](https://docs.basis-ai.com/guides/explainability-and-fairness/api-references) for more details.

In [ ]:
analyzer = ModelAnalyzer(model, "credit_risk", model_type=ModelTypes.TREE)

# To set test data used for calculating explainability
analyzer.test_features(x_test)
analyzer.stored_sample_limit(sample_limit=None)  # To output all SHAP values for the test data

# To set fairness configuration, groundtruth labels and model inference results for generating fairness metrics
analyzer.fairness_config(CONFIG_FAI).test_labels(y_test).test_inference(y_pred)

(shap_values, base_values, global_xai_data, fairness_metrics) = analyzer.analyze()

## Analyzer outputs

### SHAP values
Array of SHAP values calculated on provided `test_features`

In [ ]:
shap_values

### Base value
Base value is the average model output calculated on background data

In [ ]:
base_values

### Global XAI data
Global XAI returns the mean absolute shap values of each feature, together with its correlation with said feature

In [ ]:
global_xai_data.keys()

In [ ]:
global_xai_data["class 1"]["EXT_SOURCE_1"]

### Fairness metrics

Since fairness config is passed, the model analyzer also returns the fairness metrics.

In [ ]:
fairness_metrics.keys()

In [ ]:
fairness_df, confusion_matrices = fairness_metrics["CODE_GENDER"]["class 1"]

In [ ]:
fairness_df

In [ ]:
confusion_matrices

## Optional: Plotting explainability

Using the compute SHAP values and `shap.summary_plot`, we can obtain a preliminary view of the effects of the top features.

In [ ]:
import shap

# We use the SHAP values that correspond to `TARGET=1`, ie shap_values[1]
shap.summary_plot(shap_values[1], x_test, plot_type="bar", max_display=10)
shap.summary_plot(shap_values[1], x_test, max_display=10)